# Track an Experiment with MLflow

In this exercise, you will go through the process of training a machine learning model and apply MLOps principles to manage and monitor your machine learning pipeline.

Since you have already trained initial models (Logistic Regression and Random Forest) in a previous exercise, you will track these experiments using MLflow to monitor model performance over multiple runs.

You will need:
- `MLflow` installed (`pip install mlflow`).

### Objective

By the end of this exercise, you will be able to set up MLflow for experiment tracking, logging model metrics, comparing multiple runs, and deploying a model using MLflow's capabilities.


In [1]:
pip install mlflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Step 1: Set Up MLflow and Required Libraries

**Task:** Import the necessary libraries for MLflow and scikit-learn.

- Import libraries such as `mlflow`, `mlflow.sklearn`, `pandas`, and others you think are necessary


In [19]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from mlflow.models import infer_signature
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os




### Step 2: Create and Track an Experiment with MLflow

**Task:** Define a new experiment in MLflow.

- What command should you use to set up a new experiment in MLflow?
- Set up an experiment with the name "Wearable_Device_Stress_Classifier"


In [34]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Wearable_Device_Stress_Classifier")
mlflow.end_run()  


file_name = "C:\\Users\\niman\\OneDrive\\Desktop\\TP_Process\\data\\Device_Dataset.csv"


df = pd.read_csv(file_name)
with mlflow.start_run():
    mlflow.log_param("data_shape", df.shape)
    mlflow.log_param("columns", list(df.columns))
    print("Dataset information logged to MLflow.")
    print(df.head()) 

2024/11/09 16:50:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run stately-shoat-608 at: http://127.0.0.1:5000/#/experiments/847870391408369880/runs/b62f307c06684c70badb0f09fd958c99.
2024/11/09 16:50:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/847870391408369880.
2024/11/09 16:50:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run respected-fish-491 at: http://127.0.0.1:5000/#/experiments/847870391408369880/runs/13183eccfa28442b9c571d02791d4e29.
2024/11/09 16:50:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/847870391408369880.


Dataset information logged to MLflow.
  Activity Type  Heart Rate (BPM)  Body Temperature (°C)  \
0       walking         66.681663              37.236851   
1       cycling         80.490124              37.070160   
2       walking         70.650875              36.507567   
3       cycling         92.427842              36.183344   
4       running         97.867898              36.897490   

   Acceleration X (m/s²)  Acceleration Y (m/s²)  Acceleration Z (m/s²)  \
0              -0.675178              -1.907808              -0.863494   
1              -0.144519              -0.860385              -0.031203   
2              -0.792420              -0.413606               0.018017   
3              -0.307962               1.887688               0.472630   
4              -1.893615               0.556553              -1.366858   

   Stressed State  
0           False  
1           False  
2            True  
3            True  
4            True  


**Task:** Track training metrics and log model parameters.

- How would you start an MLflow run to log your experiment details?
- Train a Logistic Regression model and track its metrics (e.g., accuracy, precision, recall, F1 score) using MLflow.
- Log the model using `mlflow.sklearn.log_model()`.

**Hint:** You should use `with mlflow.start_run():` to start an MLflow run.


In [32]:
params = {
    "solver": "saga",
    "C": 1.0,
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

In [ ]:

def get_data(file_name):
    """Prepares data by reading, encoding categorical variables, and splitting into train/test sets."""
    df = pd.read_csv(file_name)
    X = df.drop(columns=['Stressed State'])
    y = df['Stressed State']
    
    
    encoder = OneHotEncoder(sparse_output=False)
    activity_encoded = encoder.fit_transform(X[['Activity Type']])
    activity_encoded_df = pd.DataFrame(activity_encoded, columns=encoder.get_feature_names_out(['Activity Type']))
    
    
    X = pd.concat([X.drop(columns=['Activity Type']), activity_encoded_df], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

In [6]:
def train_model(X_train, y_train, params):
    """Trains a Logistic Regression model with given parameters."""
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    return model


def evaluate_model(model, X_test, y_test):
    """Generates predictions and calculates evaluation metrics."""
    y_pred = model.predict(X_test)
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred)
    }
    return y_pred, metrics

In [ ]:
def log_model_to_mlflow(model, params, metrics, X_train, registered_model_name="Wearable_Stress_Model"):
    """Logs parameters, metrics, and the model to MLflow."""
    with mlflow.start_run():
        
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        
        
        mlflow.set_tag("Model Type", "Logistic Regression")
        signature = infer_signature(X_train, model.predict(X_train))
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="logistic_regression_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=registered_model_name
        )
        print("Model and metrics successfully logged to MLflow.")

In [11]:
X_train, X_test, y_train, y_test = get_data("C:\\Users\\niman\\OneDrive\\Desktop\\TP_Process\\data\\Device_Dataset.csv")

In [ ]:
model = train_model(X_train, y_train, params)
y_pred, metrics = evaluate_model(model, X_test, y_test)


log_model_to_mlflow(model, params, metrics, X_train)


c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
Registered model 'Wearable_Stress_Model' already exists. Creating a new version of this model...
2024/11/09 12:55:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Wearable_Stress_Model, version 6
Created version '6' of model 'Wearable_Stress_Model'.
2024/11/09 12:55:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run skittish-wolf-222 at: http://127.0.0.1:5000/#/experiments/847870391408369880/runs/ba7da0fc99ac42dd9814eac667d17ad2.
2024/11/09 12:55:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/847870391408369880.


Model and metrics successfully logged to MLflow.


### Step 3: Compare Multiple Runs

- **Task:** Modify the model parameters (e.g., change solver type or regularization strength) and re-run the experiment log.
- **Open the MLflow UI:** Start the MLflow UI by running:
  ```bash
  mlflow ui
  ```
- Navigate to `http://127.0.0.1:5000` to compare different experiment runs visually. Observe how changing parameters affects the metrics.


In [29]:

param_sets = [
    {"solver": "liblinear", "C": 1.0, "max_iter": 1000, "multi_class": "auto", "random_state": 8888},
    {"solver": "liblinear", "C": 0.5, "max_iter": 1000, "multi_class": "auto", "random_state": 8888},
    {"solver": "saga", "C": 1.0, "max_iter": 1000, "multi_class": "auto", "random_state": 8888},
    {"solver": "saga", "C": 0.5, "max_iter": 1000, "multi_class": "auto", "random_state": 8888}
]


for params in param_sets:
    with mlflow.start_run():
        
        model = LogisticRegression(**params)
        model.fit(X_train, y_train)
        
        
        y_pred = model.predict(X_test)
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1_score": f1_score(y_test, y_pred)
        }
        
        
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        
        
        mlflow.set_tag("Model Type", "Logistic Regression")
        
        
        signature = infer_signature(X_train, model.predict(X_train))
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="logistic_regression_model",
            signature=signature,
            input_example=X_train,
            registered_model_name="Wearable_Stress_Model"
        )
        
        print(f"Run with parameters {params} completed. Metrics: {metrics}")




c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
Registered model 'Wearable_Stress_Model' already exists. Creating a new version of this model...
2024/11/09 12:36:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Wearable_Stress_Model, version 2
Created version '2' of model 'Wearable_Stress_Model'.
2024/11/09 12:36:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run glamorous-bug-629 at: http://127.0.0.1:5000/#/experiments/847870391408369880/runs/5c9b8d56b0764c4a9195e62507498d67.
2024/11/09 12:36:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/847870391408369880.


Run with parameters {'solver': 'liblinear', 'C': 1.0, 'max_iter': 1000, 'multi_class': 'auto', 'random_state': 8888} completed. Metrics: {'accuracy': 0.64, 'precision': np.float64(0.7014925373134329), 'recall': np.float64(0.47474747474747475), 'f1_score': np.float64(0.5662650602409639)}


c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
Registered model 'Wearable_Stress_Model' already exists. Creating a new version of this model...
2024/11/09 12:36:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Wearable_Stress_Model, version 3
Created version '3' of model 'Wearable_Stress_Model'.
2024/11/09 12:36:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run bedecked-calf-204 at: http://127.0.0.1:5000/#/experiments/847870391408369880/runs/3ba07268c3e84199ac0124715e9da49d.
2024/11/09 12:36:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/847870391408369880.


Run with parameters {'solver': 'liblinear', 'C': 0.5, 'max_iter': 1000, 'multi_class': 'auto', 'random_state': 8888} completed. Metrics: {'accuracy': 0.64, 'precision': np.float64(0.7014925373134329), 'recall': np.float64(0.47474747474747475), 'f1_score': np.float64(0.5662650602409639)}


c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
Registered model 'Wearable_Stress_Model' already exists. Creating a new version of this model...
2024/11/09 12:36:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Wearable_Stress_Model, version 4
Created version '4' of model 'Wearable_Stress_Model'.
2024/11/09 12:36:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run persistent-hound-108 at: http://127.0.0.1:5000/#/experiments/847870391408369880/run

Run with parameters {'solver': 'saga', 'C': 1.0, 'max_iter': 1000, 'multi_class': 'auto', 'random_state': 8888} completed. Metrics: {'accuracy': 0.64, 'precision': np.float64(0.7014925373134329), 'recall': np.float64(0.47474747474747475), 'f1_score': np.float64(0.5662650602409639)}


c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
Registered model 'Wearable_Stress_Model' already exists. Creating a new version of this model...
2024/11/09 12:36:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Wearable_Stress_Model, version 5
Created version '5' of model 'Wearable_Stress_Model'.
2024/11/09 12:36:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run caring-mouse-386 at: http://127.0.0.1:5000/#/experiments/847870391408369880/runs/7c

Run with parameters {'solver': 'saga', 'C': 0.5, 'max_iter': 1000, 'multi_class': 'auto', 'random_state': 8888} completed. Metrics: {'accuracy': 0.64, 'precision': np.float64(0.7014925373134329), 'recall': np.float64(0.47474747474747475), 'f1_score': np.float64(0.5662650602409639)}


### Step 4: Log additional artifacts

**Task:** Log a confusion matrix as an artifact in MLflow.

- How would you log a confusion matrix plot to MLflow?
- Plot a confusion matrix for your predictions and log it as an artifact.

**Hint:** Use `seaborn` for plotting and `mlflow.log_artifact()` for logging the plot.


In [8]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def plot_and_log_confusion_matrix(y_test, y_pred):
    """Generates and logs a confusion matrix plot as an artifact in MLflow."""
    cm = confusion_matrix(y_test, y_pred)
    
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Not Stressed", "Stressed"], yticklabels=["Not Stressed", "Stressed"])
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")

    
    os.makedirs("plots", exist_ok=True)
    plot_path = "plots/confusion_matrix.png"
    plt.savefig(plot_path)
    plt.close()

   
    mlflow.log_artifact(plot_path)
    print(f"Confusion matrix logged as artifact: {plot_path}")

In [35]:
model = train_model(X_train, y_train, params)
y_pred, metrics = evaluate_model(model, X_test, y_test)


log_model_to_mlflow(model, params, metrics, X_train)
plot_and_log_confusion_matrix(y_test, y_pred)

c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\niman\OneDrive\Desktop\TP_Process\env\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
Registered model 'Wearable_Stress_Model' already exists. Creating a new version of this model...
2024/11/09 16:50:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Wearable_Stress_Model, version 14
Created version '14' of model 'Wearable_Stress_Model'.
2024/11/09 16:50:42 INFO mlflow.tracking._tracking_service.client: 🏃 View run nervous-perch-464 at: http://127.0.0.1:5000/#/experiments/847870391408369880/runs

Model and metrics successfully logged to MLflow.
Confusion matrix logged as artifact: plots/confusion_matrix.png


### Step 5: Model packaging with MLflow

**Task:** Register the trained model in the MLflow Model Registry.

- How can you register the best model for versioning and potential deployment?
- Use the `mlflow.register_model()` function to add your model to the registry.



In [ ]:
model_uri = f"runs:/{run.info.run_id}/logistic_regression_model"
registered_model_name = "Wearable_Stress_Model"

    
registered_model = mlflow.register_model(model_uri=model_uri, name=registered_model_name)
    
print(f"Model registered with name: {registered_model_name} and version: {registered_model.version}")

### Step 6: Deploy the model as a REST API

**Task:** Deploy the registered model as a REST API using MLflow.

- How would you serve the model locally?
- Use `mlflow models serve` to start a local REST API.

```bash
mlflow models serve -m "models:/WearableStressClassifierModel/1" -p 1234
```

**Task:** Make a prediction request to your model's REST API.

- Use Python's `requests` library to send a JSON request to the REST API and get predictions.


In [ ]:
import requests
import json

value1 = 70  
value2 = 36  
value3 = 1   
value4 = 1   
value5 = 0   
value6 = 0   
value7 = 0   
value8 = 1   


data = {
    "dataframe_records": [
        {
            "Heart Rate (BPM)": value1,
            "Body Temperature (°C)": value2,
            "Acceleration X (m/s²)": value3,
            "Acceleration Y (m/s²)": value4,
            "Acceleration Z (m/s²)": value5,
            "Activity Type_cycling": value6,
            "Activity Type_running": value7,
            "Activity Type_walking": value8
        }
    ]
}


response = requests.post("http://localhost:1234/invocations", json=data)


print("Prediction:", response.json())

Prediction: {'predictions': [False]}


### Step 7: Manage model versions

**Task:** Explore the MLflow Model Registry.

- How can you manage different versions of your model in MLflow?
- Experiment with updating the registered model after retraining with different hyperparameters.


In [ ]:
from mlflow.tracking import MlflowClient
import mlflow.pyfunc
import pandas as pd


client = MlflowClient()


client.update_model_version(
    name="Wearable_Stress_Model",
    version=12,
    description="Trained with updated parameters"
)


model_version_uri = "models:/Wearable_Stress_Model/12"
model = mlflow.pyfunc.load_model(model_version_uri)


data = pd.DataFrame([{
    "Heart Rate (BPM)": 70.0,
    "Body Temperature (°C)": 36.0,
    "Acceleration X (m/s²)": 1.0,
    "Acceleration Y (m/s²)": 1.0,
    "Acceleration Z (m/s²)": 0.0,
    "Activity Type_cycling": 0.0,
    "Activity Type_running": 0.0,
    "Activity Type_walking": 1.0
}])


predictions = model.predict(data)


print("Predictions:", predictions)


Predictions: [False]


### Step 8: Automated workflow

**Task:** Use MLflow Projects to automate the workflow.

- Create an `MLproject` file to define the script and dependencies in a reproducible manner.
- Run the project locally to validate your setup.

```bash
mlflow run .
```


### Step 9: Visualize metrics and monitor models

**Task:** Visualize metrics using the MLflow UI.

- How can you use the MLflow UI to compare runs and visualize metrics like `accuracy`, `precision`, `recall`, and `f1_score`?
- Set up custom monitoring dashboards using Grafana if needed.


### Summary and exploration

- **Key Questions:**
  1. How did changing model parameters impact performance metrics like accuracy, precision, and recall?
  2. What are the benefits of tracking multiple runs using MLflow?
  3. How can artifact logging be useful for diagnosing model behavior?
  4. What are the challenges in deploying machine learning models, and how does MLflow assist?
  5. How can visualizing metrics help you understand your model's performance over time?
  6. Why is monitoring model drift important, and how can MLflow help?


1. I don't see much of difference between models on changing the solvers and iterations

2. It helps us to compare the results of multiple models by changing their parameters 

3. By logging the visulisation and lggoing data snapshots helps to store and later analyze the models and diagnose the problems

4. We track the different versions of the model and experiments , the models are reproductible and allows us to monitor the models

5. Visualizing metrics over time reveals trends like drift, overfitting, or metric trade-offs, helping you monitor model health, set performance alerts, and fine-tune parameters to ensure reliable performance.

6. Monitoring model drift is crucial to catch declines in model performance as data patterns change. MLflow helps by logging and tracking metrics over time, allowing you to spot drift early and update models quickly to maintain accuracy.